# Wires från dgn till gdb
## fas och topp för sig
## littera som attribut

In [4]:
# Läs settings
import json

settings_file = r"Q:\Projekt\Data_2024\styrfiler\settings_SEKNNO.json"

with open(settings_file, 'r', encoding='utf-8') as file:
    settings = json.load(file)

In [5]:
settings

{'run_ID': '2501yy', 'powerline_list': 'Q:/Projekt/Data_2024/styrfiler/veg_kanttrad_2501yy.txt', 'scandate_file': 'Q:/Projekt/Data_2024/skanningsdatum/skanningsdatum_2501yy.txt', 'local_folder': 'C:/SVK_2024/pythonkörningar', 'working_gdb_template': 'Q:/Projekt/Data_2024/working_template.gdb', 'results_gdb_template': 'Q:/Projekt/Data_2024/results_template.gdb', 'wires_gdb_template': 'Q:/Projekt/Data_2024/wires_template.gdb', 'dgn_folder': 'Q:/Projekt/Analys_2024/DGN_2024', 'powerlines_folder': 'Q:/Projekt/Analys_2024/ledningar', 'domains_folder': 'Q:/Projekt/Data_2024/Underlag_SVK/Domains', 'LG_polygons': 'Q:/Projekt/Data_2024/Underlag_SVK/Bestallningsunderlag_2024.gdb/GNG_LEDNINGSGATA', 'station_polygons': 'Q:/Projekt/Data_2024/Underlag_SVK/Bestallningsunderlag_2024.gdb/GNG_STATIONSOMRADE'}

In [6]:
import arcpy, os, glob
import pandas as pd
import shutil

run_ID = settings["run_ID"]
powerline_list = settings["powerline_list"]
local_dir = settings["local_folder"]
dgn_dir = settings["dgn_folder"]
wires_gdb_template = settings["wires_gdb_template"]
domains_folder = settings["domains_folder"]

cvd_LEDNINGSGATA_path = os.path.join(domains_folder, "cvd_LEDNINGSGATA.txt")
wires_gdb = os.path.join(local_dir, f"wires_{run_ID}.gdb")

# Create wire gdb as copy of template
if os.path.exists(wires_gdb):
    print(f"{wires_gdb} finns redan")
else: 
    shutil.copytree(wires_gdb_template, wires_gdb)

arcpy.env.overwriteOutput = True

def wires_to_gdb(row):
    # Ange i definitionen att row är en pandas serie
    LG = row["LG"]
    line = row["line"]
    littera = row["Littera"]
    LG_code = cvd_LEDNINGSGATA[LG]
    
    print(f"Påbörjar export av fas- och topplinor för {LG}_{line}, {littera}")
    
    dgn = os.path.join(dgn_dir, f"{LG}_{line}.dgn")
    arcpy.env.workspace = dgn_dir # Varför ska den ha dgn dir som workspace?
    
    # faslinor
    fc_fas = f"{LG}_{line}_fas"
    # Create a value table that will hold the input feature classes for Merge
    vTab_fas = arcpy.ValueTable()
    layer_fas = "Layer_fas"
    arcpy.Delete_management(layer_fas)
    arcpy.MakeFeatureLayer_management(dgn + '/Polyline', layer_fas, "\"Level\" = 1")
    vTab_fas.addRow(layer_fas)
        
    # topplinor
    fc_topp = f"{LG}_{line}_topp"
    # Create a value table that will hold the input feature classes for Merge
    vTab_topp = arcpy.ValueTable()
    layer_topp = "Layer_topp"
    arcpy.Delete_management(layer_topp)
    arcpy.MakeFeatureLayer_management(dgn + '/Polyline', layer_topp, "\"Level\" = 2")
    vTab_topp.addRow(layer_topp)
    
    # Merge the CAD features into one feature class
    arcpy.env.workspace = wires_gdb
    
    # faslinor
    arcpy.Merge_management(vTab_fas, fc_fas)
    # Add and populate littera field
    arcpy.management.AddField(fc_fas, "Littera", "TEXT")
    arcpy.management.CalculateField(fc_fas, "Littera", "".join(("'",littera,"'")))
    arcpy.management.AddField(fc_fas, "LG", "LONG")
    arcpy.management.CalculateField(fc_fas, "LG", int(LG_code))
    arcpy.management.AddField(fc_fas, "lintyp", "TEXT")
    arcpy.management.CalculateField(fc_fas, "lintyp", "'ledare'")
    # domains
    arcpy.management.AssignDomainToField(fc_fas, "LG", "cvd_LEDNINGSGATA")
    arcpy.management.AssignDomainToField(fc_fas, "Littera", "cvd_LITTERA_LEDNING")
    
    # ta bort fält som inte ska användas DETTA BEHÖVER GÖRAS PÅ BÅDE FAS OCH TOPP MEN NUVARANDE 
    # UPPREPNING ÄR KLUMPIG
    keep_fields = ["LG", "Littera", "lintyp", "Shape", "Shape_Length", "OBJECTID"]
    all_fields = arcpy.ListFields(fc_fas)
    fields_to_delete = [field.name for field in all_fields if field.name not in keep_fields]
    if fields_to_delete:
        arcpy.DeleteField_management(fc_fas, fields_to_delete)   

    # Räkna faslinor, varna om inga finns
    arcpy.management.DefineProjection(fc_fas, sr)
    result = arcpy.management.GetCount(fc_fas)
    count = int(result[0])
    if count == 0:
        print(f"VARNING: Inga faslinor hittades i level 1 i {LG}_{line}.dgn")
    
    # topplinor
    arcpy.Merge_management(vTab_topp, fc_topp)
    # Add and populate littera field
    arcpy.management.AddField(fc_topp, "Littera", "TEXT")
    arcpy.management.CalculateField(fc_topp, "Littera", "".join(("'",littera,"'")))
    arcpy.management.AddField(fc_topp, "LG", "LONG")
    arcpy.management.CalculateField(fc_topp, "LG", int(LG_code))
    arcpy.management.AddField(fc_topp, "lintyp", "TEXT")
    arcpy.management.CalculateField(fc_topp, "lintyp", "'topplina'")
    # domains
    arcpy.management.AssignDomainToField(fc_fas, "LG", "cvd_LEDNINGSGATA")
    arcpy.management.AssignDomainToField(fc_fas, "Littera", "cvd_LITTERA_LEDNING")
    
    # ta bort fält som inte ska användas
    keep_fields = ["LG", "Littera", "lintyp", "Shape", "Shape_Length", "OBJECTID"]
    all_fields = arcpy.ListFields(fc_topp)
    fields_to_delete = [field.name for field in all_fields if field.name not in keep_fields]
    if fields_to_delete:
        arcpy.DeleteField_management(fc_topp, fields_to_delete)
    
    arcpy.management.DefineProjection(fc_topp, sr)

    # Räkna topplinor, varna om inga finns
    result = arcpy.management.GetCount(fc_topp)
    count = int(result[0])
    if count == 0:
        print(f"VARNING: Inga topplinor hittades i level 2 i {LG}_{line}.dgn")
        
    # Städa bort tillfälliga lager från minnet
    arcpy.env.workspace = dgn_dir
    arcpy.Delete_management(layer_fas)
    arcpy.Delete_management(layer_topp)
        
# Sätt detta i en main-funktion för tydlighet?
sr = arcpy.SpatialReference("SWEREF99_TM", "RH2000")
powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
df_cvd_LEDNINGSGATA = pd.read_csv(cvd_LEDNINGSGATA_path, sep="\t", header=0)
cvd_LEDNINGSGATA = {df_cvd_LEDNINGSGATA.Description[i]: df_cvd_LEDNINGSGATA.Code[i] for i in range(len(df_cvd_LEDNINGSGATA))}

powerlines_df.apply(wires_to_gdb, axis=1)


C:/SVK_2024/pythonkörningar\wires_2501yy.gdb finns redan
Påbörjar export av fas- och topplinor för LG021_1, AL6 S7
Påbörjar export av fas- och topplinor för LG021_2, UL6 S5-7
Påbörjar export av fas- och topplinor för LG025_1, AL3 S8
Påbörjar export av fas- och topplinor för LG025_2, AL3 S5
Påbörjar export av fas- och topplinor för LG025_3, AL3 S3
Påbörjar export av fas- och topplinor för LG025_4, AL3 S4
Påbörjar export av fas- och topplinor för LG028_9, AL1 S1
Påbörjar export av fas- och topplinor för LG028_10, AL2 S1-2
Påbörjar export av fas- och topplinor för LG029_1, AL3 S1
Påbörjar export av fas- och topplinor för LG029_2, AL6 S3
Påbörjar export av fas- och topplinor för LG029_3, AL6 S4
Påbörjar export av fas- och topplinor för LG029_4, AL6 S7
Påbörjar export av fas- och topplinor för LG029_5, CL14 S1-2
Påbörjar export av fas- och topplinor för LG029_6, CL4 S1-3
Påbörjar export av fas- och topplinor för LG029_7, CL9 S5
Påbörjar export av fas- och topplinor för LG029_8, UL6 S8
Påbör

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
dtype: object

In [7]:
arcpy.env.workspace = wires_gdb

feature_classes = arcpy.ListFeatureClasses()
print(f"Antal featureklasser: {len(feature_classes)}")

for fc in feature_classes:
    feature_count = int(arcpy.management.GetCount(fc)[0])
    print(f"{fc}: {feature_count} features")

Antal featureklasser: 52
LG021_1_fas: 6 features
LG021_1_topp: 4 features
LG021_2_fas: 765 features
LG021_2_topp: 508 features
LG025_1_fas: 33 features
LG025_1_topp: 16 features
LG025_2_fas: 333 features
LG025_2_topp: 117 features
LG025_3_fas: 129 features
LG025_3_topp: 55 features
LG025_4_fas: 372 features
LG025_4_topp: 8 features
LG028_9_fas: 984 features
LG028_9_topp: 654 features
LG028_10_fas: 33 features
LG028_10_topp: 19 features
LG029_1_fas: 9 features
LG029_1_topp: 6 features
LG029_2_fas: 12 features
LG029_2_topp: 8 features
LG029_3_fas: 163 features
LG029_3_topp: 68 features
LG029_4_fas: 30 features
LG029_4_topp: 20 features
LG029_5_fas: 581 features
LG029_5_topp: 252 features
LG029_6_fas: 6 features
LG029_6_topp: 4 features
LG029_7_fas: 105 features
LG029_7_topp: 68 features
LG029_8_fas: 18 features
LG029_8_topp: 12 features
LG032_6_fas: 411 features
LG032_6_topp: 274 features
LG032_7_fas: 207 features
LG032_7_topp: 20 features
LG032_8_fas: 24 features
LG032_8_topp: 16 featur